<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hotel-Cancel-Culture---EDA-Notebook" data-toc-modified-id="Hotel-Cancel-Culture---EDA-Notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><strong>Hotel Cancel Culture</strong> - <strong>EDA Notebook</strong></a></span></li><li><span><a href="#Import-Packages" data-toc-modified-id="Import-Packages-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><strong>Import Packages</strong></a></span></li><li><span><a href="#Read-Data" data-toc-modified-id="Read-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><strong>Read Data</strong></a></span></li><li><span><a href="#Identifying-Target-Feature" data-toc-modified-id="Identifying-Target-Feature-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><strong>Identifying Target Feature</strong></a></span></li><li><span><a href="#Visualizing-the-Data" data-toc-modified-id="Visualizing-the-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Visualizing the Data</a></span></li><li><span><a href="#Reviewing-Statistics" data-toc-modified-id="Reviewing-Statistics-6"><span class="toc-item-num">6&nbsp;&nbsp;</span><strong>Reviewing Statistics</strong></a></span><ul class="toc-item"><li><span><a href="#Dropping-Company-Column" data-toc-modified-id="Dropping-Company-Column-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Dropping <code>Company</code> Column</a></span></li><li><span><a href="#Inspecting-the-Agent-Feature" data-toc-modified-id="Inspecting-the-Agent-Feature-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Inspecting the <code>Agent</code> Feature</a></span></li><li><span><a href="#Comparing-Market_Segment-and-Distribution_Channel" data-toc-modified-id="Comparing-Market_Segment-and-Distribution_Channel-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Comparing <code>Market_Segment</code> and <code>Distribution_Channel</code></a></span></li><li><span><a href="#Filling-Missing-Values-for-Country-and-Children" data-toc-modified-id="Filling-Missing-Values-for-Country-and-Children-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Filling Missing Values for <code>Country</code> and <code>Children</code></a></span></li></ul></li><li><span><a href="#Inspecting-Feature-Data-Types" data-toc-modified-id="Inspecting-Feature-Data-Types-7"><span class="toc-item-num">7&nbsp;&nbsp;</span><strong>Inspecting Feature Data Types</strong></a></span><ul class="toc-item"><li><span><a href="#Converting-to-String" data-toc-modified-id="Converting-to-String-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Converting to String</a></span></li><li><span><a href="#Converting-to-DateTime" data-toc-modified-id="Converting-to-DateTime-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Converting to DateTime</a></span></li></ul></li><li><span><a href="#EDA---Features" data-toc-modified-id="EDA---Features-8"><span class="toc-item-num">8&nbsp;&nbsp;</span><strong>EDA - Features</strong></a></span></li><li><span><a href="#--->-🛑-FIX:-Update-narrative-per-feature" data-toc-modified-id="--->-🛑-FIX:-Update-narrative-per-feature-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>-- &gt; 🛑 <strong>FIX</strong>: Update narrative per feature</a></span><ul class="toc-item"><li><span><a href="#Toggle-Visualizations" data-toc-modified-id="Toggle-Visualizations-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span><strong>Toggle Visualizations</strong></a></span></li><li><span><a href="#Reservation_Status" data-toc-modified-id="Reservation_Status-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span><code>Reservation_Status</code></a></span></li><li><span><a href="#Is_Canceled" data-toc-modified-id="Is_Canceled-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span><code>Is_Canceled</code></a></span></li><li><span><a href="#Lead_Time" data-toc-modified-id="Lead_Time-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span><code>Lead_Time</code></a></span></li><li><span><a href="#Arrival_Date_Year" data-toc-modified-id="Arrival_Date_Year-9.5"><span class="toc-item-num">9.5&nbsp;&nbsp;</span><code>Arrival_Date_Year</code></a></span></li><li><span><a href="#Stays_in_Weekend_Nights" data-toc-modified-id="Stays_in_Weekend_Nights-9.6"><span class="toc-item-num">9.6&nbsp;&nbsp;</span><code>Stays_in_Weekend_Nights</code></a></span></li><li><span><a href="#Stays_in_Week_Nights" data-toc-modified-id="Stays_in_Week_Nights-9.7"><span class="toc-item-num">9.7&nbsp;&nbsp;</span><code>Stays_in_Week_Nights</code></a></span></li><li><span><a href="#Adults" data-toc-modified-id="Adults-9.8"><span class="toc-item-num">9.8&nbsp;&nbsp;</span><code>Adults</code></a></span></li><li><span><a href="#Children" data-toc-modified-id="Children-9.9"><span class="toc-item-num">9.9&nbsp;&nbsp;</span><code>Children</code></a></span></li><li><span><a href="#Babies" data-toc-modified-id="Babies-9.10"><span class="toc-item-num">9.10&nbsp;&nbsp;</span><code>Babies</code></a></span></li><li><span><a href="#Meal" data-toc-modified-id="Meal-9.11"><span class="toc-item-num">9.11&nbsp;&nbsp;</span><code>Meal</code></a></span></li><li><span><a href="#Country" data-toc-modified-id="Country-9.12"><span class="toc-item-num">9.12&nbsp;&nbsp;</span><code>Country</code></a></span></li><li><span><a href="#Market_Segment" data-toc-modified-id="Market_Segment-9.13"><span class="toc-item-num">9.13&nbsp;&nbsp;</span><code>Market_Segment</code></a></span></li><li><span><a href="#Is_Repeated_Guest" data-toc-modified-id="Is_Repeated_Guest-9.14"><span class="toc-item-num">9.14&nbsp;&nbsp;</span><code>Is_Repeated_Guest</code></a></span></li><li><span><a href="#Previous_Cancellations" data-toc-modified-id="Previous_Cancellations-9.15"><span class="toc-item-num">9.15&nbsp;&nbsp;</span><code>Previous_Cancellations</code></a></span></li><li><span><a href="#Previous_Bookings_Not_Canceled" data-toc-modified-id="Previous_Bookings_Not_Canceled-9.16"><span class="toc-item-num">9.16&nbsp;&nbsp;</span><code>Previous_Bookings_Not_Canceled</code></a></span></li><li><span><a href="#Reserved_Room_Type" data-toc-modified-id="Reserved_Room_Type-9.17"><span class="toc-item-num">9.17&nbsp;&nbsp;</span><code>Reserved_Room_Type</code></a></span></li><li><span><a href="#Assigned_Room_Type" data-toc-modified-id="Assigned_Room_Type-9.18"><span class="toc-item-num">9.18&nbsp;&nbsp;</span><code>Assigned_Room_Type</code></a></span></li><li><span><a href="#Booking_Changes" data-toc-modified-id="Booking_Changes-9.19"><span class="toc-item-num">9.19&nbsp;&nbsp;</span><code>Booking_Changes</code></a></span></li><li><span><a href="#Deposit_Type" data-toc-modified-id="Deposit_Type-9.20"><span class="toc-item-num">9.20&nbsp;&nbsp;</span><code>Deposit_Type</code></a></span></li><li><span><a href="#Days_in_Waiting_List" data-toc-modified-id="Days_in_Waiting_List-9.21"><span class="toc-item-num">9.21&nbsp;&nbsp;</span><code>Days_in_Waiting_List</code></a></span></li><li><span><a href="#Customer_Type" data-toc-modified-id="Customer_Type-9.22"><span class="toc-item-num">9.22&nbsp;&nbsp;</span><code>Customer_Type</code></a></span></li><li><span><a href="#ADR" data-toc-modified-id="ADR-9.23"><span class="toc-item-num">9.23&nbsp;&nbsp;</span><code>ADR</code></a></span></li><li><span><a href="#Required_Car_Parking_Spaces" data-toc-modified-id="Required_Car_Parking_Spaces-9.24"><span class="toc-item-num">9.24&nbsp;&nbsp;</span><code>Required_Car_Parking_Spaces</code></a></span></li><li><span><a href="#Total_of_Special_Requests" data-toc-modified-id="Total_of_Special_Requests-9.25"><span class="toc-item-num">9.25&nbsp;&nbsp;</span><code>Total_of_Special_Requests</code></a></span></li><li><span><a href="#Reservation_Status_Date" data-toc-modified-id="Reservation_Status_Date-9.26"><span class="toc-item-num">9.26&nbsp;&nbsp;</span><code>Reservation_Status_Date</code></a></span></li><li><span><a href="#Agent_Group" data-toc-modified-id="Agent_Group-9.27"><span class="toc-item-num">9.27&nbsp;&nbsp;</span><code>Agent_Group</code></a></span></li></ul></li><li><span><a href="#🛑-Post-EDA" data-toc-modified-id="🛑-Post-EDA-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>🛑 <strong>Post-EDA</strong></a></span><ul class="toc-item"><li><span><a href="#Updating-Meal" data-toc-modified-id="Updating-Meal-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Updating <code>Meal</code></a></span></li><li><span><a href="#Engineering-Arrival_Date" data-toc-modified-id="Engineering-Arrival_Date-10.2"><span class="toc-item-num">10.2&nbsp;&nbsp;</span>Engineering <code>Arrival_Date</code></a></span></li><li><span><a href="#Engineering-Stay-Length" data-toc-modified-id="Engineering-Stay-Length-10.3"><span class="toc-item-num">10.3&nbsp;&nbsp;</span>Engineering <code>Stay Length</code></a></span></li><li><span><a href="#Dropping-Reservation_Status" data-toc-modified-id="Dropping-Reservation_Status-10.4"><span class="toc-item-num">10.4&nbsp;&nbsp;</span>Dropping <code>Reservation_Status</code></a></span></li><li><span><a href="#Dropping-Arrival_Date_Year" data-toc-modified-id="Dropping-Arrival_Date_Year-10.5"><span class="toc-item-num">10.5&nbsp;&nbsp;</span>Dropping <code>Arrival_Date_Year</code></a></span></li><li><span><a href="#Final-Data-Review" data-toc-modified-id="Final-Data-Review-10.6"><span class="toc-item-num">10.6&nbsp;&nbsp;</span>Final Data Review</a></span></li></ul></li><li><span><a href="#Of-Pandas-and-Pickles" data-toc-modified-id="Of-Pandas-and-Pickles-11"><span class="toc-item-num">11&nbsp;&nbsp;</span><strong>Of Pandas and Pickles</strong></a></span><ul class="toc-item"><li><span><a href="#Peter-Panda-Picked-a-Peck-of-Pickled-DataFrames..." data-toc-modified-id="Peter-Panda-Picked-a-Peck-of-Pickled-DataFrames...-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Peter Panda Picked a Peck of Pickled DataFrames...</a></span></li></ul></li><li><span><a href="#Moving-to-Modeling!" data-toc-modified-id="Moving-to-Modeling!-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Moving to Modeling!</a></span></li></ul></div>

# **Hotel Cancel Culture** - **EDA Notebook**

---

**Author:** Ben McCarty

**Capstone Project** - Classification, Time Series Modeling

**Contact:** bmccarty505@gmail.com

---

---

**Who?**
>* 🏢 **Revenue Management (RM) teams** for hotel groups (corporate, franchise)
>
>
>* 🏨 On-site GMs, Sales, and Ops teams

---

**Why?**
>* 💰 **Revenue Management:** 
>  * Revenue optimization: Right price, right time, right customer
>    * Dynamic pricing
>    * Distribution channels
>    * Pricing per room type
>
>
>* 🤝 **Sales:**
>  * Group sales (pickup/wash)
>  * BT (performance/company for both GPP and LNR rates)
>
>
>* 🛌 **Rooms Ops:**
>  * Forecasting occupancy, arrivals, departures, stay-overs, same-day booking demand, and probability of guest relocation in the case of oversell.
>  * Determining staff schedules and periods of high demand
>
>
>* 🍰 ☕ **Food and Beverage:**
>  * Ordering food/supplies overall
>  * Scheduling staff
>  * Determining busy times (breakfast, lunch, dinner)
>    * Staffing, specific food/supplies

---

**What?**
>* 🧾 Dataset comprised of... 
>  * 32 different features
>    * Detailed explanation of features (and sub-categories, when appropriate) available in Readme
>  * Nearly 120,000 reservation records
>  * Source cited in Readme

---

 **How?**
>* Which models/methods?
>  * 🔢 Classifiers 🌳
    * XGBoost, RFC, ABC, etc.
>  * ⏳ Time Series Analysis 📈
    * PMD auto-arima
    * Statsmodels vector autoregression
>
>
>* Data prep and feature engineering

---

---

> **Goal:** To prepare data for classification modeling in next notebook.
>
>
> **Purpose:** to explore, clean, and organize.
>
>
> **Process:**
>
>    * Inspecting data integrity and statistics
>    * Splitting data by hotel type ("City" vs. "Resort")
>    * Filling any missing values
>    * Save processed data for modeling notebook
>
>
> **Modeling Notebook:**
>
>    * Performing train/test split
>    * Training the model
>    * Evaluate performance metrics
>    * Provide final recommendations

---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hotel-Cancel-Culture---EDA-Notebook" data-toc-modified-id="Hotel-Cancel-Culture---EDA-Notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><strong>Hotel Cancel Culture</strong> - <strong>EDA Notebook</strong></a></span></li><li><span><a href="#Import-Packages" data-toc-modified-id="Import-Packages-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><strong>Import Packages</strong></a></span></li><li><span><a href="#Read-Data" data-toc-modified-id="Read-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><strong>Read Data</strong></a></span></li><li><span><a href="#Identifying-Target-Feature" data-toc-modified-id="Identifying-Target-Feature-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><strong>Identifying Target Feature</strong></a></span></li><li><span><a href="#Reviewing-Statistics" data-toc-modified-id="Reviewing-Statistics-5"><span class="toc-item-num">5&nbsp;&nbsp;</span><strong>Reviewing Statistics</strong></a></span><ul class="toc-item"><li><span><a href="#Dropping-Company-Column" data-toc-modified-id="Dropping-Company-Column-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Dropping <code>Company</code> Column</a></span></li><li><span><a href="#Inspecting-the-Agent-Feature" data-toc-modified-id="Inspecting-the-Agent-Feature-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Inspecting the <code>Agent</code> Feature</a></span></li><li><span><a href="#Comparing-Market_Segment-and-Distribution_Channel" data-toc-modified-id="Comparing-Market_Segment-and-Distribution_Channel-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Comparing <code>Market_Segment</code> and <code>Distribution_Channel</code></a></span></li><li><span><a href="#Filling-Missing-Values-for-Country-and-Children" data-toc-modified-id="Filling-Missing-Values-for-Country-and-Children-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Filling Missing Values for <code>Country</code> and <code>Children</code></a></span></li></ul></li><li><span><a href="#Inspecting-Feature-Data-Types" data-toc-modified-id="Inspecting-Feature-Data-Types-6"><span class="toc-item-num">6&nbsp;&nbsp;</span><strong>Inspecting Feature Data Types</strong></a></span><ul class="toc-item"><li><span><a href="#Converting-to-String" data-toc-modified-id="Converting-to-String-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Converting to String</a></span></li><li><span><a href="#Converting-to-DateTime" data-toc-modified-id="Converting-to-DateTime-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Converting to DateTime</a></span></li></ul></li><li><span><a href="#EDA---Features" data-toc-modified-id="EDA---Features-7"><span class="toc-item-num">7&nbsp;&nbsp;</span><strong>EDA - Features</strong></a></span></li><li><span><a href="#--->-🛑-FIX:-Update-narrative-per-feature" data-toc-modified-id="--->-🛑-FIX:-Update-narrative-per-feature-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>-- &gt; 🛑 <strong>FIX</strong>: Update narrative per feature</a></span><ul class="toc-item"><li><span><a href="#Toggle-Visualizations" data-toc-modified-id="Toggle-Visualizations-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span><strong>Toggle Visualizations</strong></a></span></li><li><span><a href="#Reservation_Status" data-toc-modified-id="Reservation_Status-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span><code>Reservation_Status</code></a></span></li><li><span><a href="#Is_Canceled" data-toc-modified-id="Is_Canceled-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span><code>Is_Canceled</code></a></span></li><li><span><a href="#Lead_Time" data-toc-modified-id="Lead_Time-8.4"><span class="toc-item-num">8.4&nbsp;&nbsp;</span><code>Lead_Time</code></a></span></li><li><span><a href="#Arrival_Date_Year" data-toc-modified-id="Arrival_Date_Year-8.5"><span class="toc-item-num">8.5&nbsp;&nbsp;</span><code>Arrival_Date_Year</code></a></span></li><li><span><a href="#Stays_in_Weekend_Nights" data-toc-modified-id="Stays_in_Weekend_Nights-8.6"><span class="toc-item-num">8.6&nbsp;&nbsp;</span><code>Stays_in_Weekend_Nights</code></a></span></li><li><span><a href="#Stays_in_Week_Nights" data-toc-modified-id="Stays_in_Week_Nights-8.7"><span class="toc-item-num">8.7&nbsp;&nbsp;</span><code>Stays_in_Week_Nights</code></a></span></li><li><span><a href="#Adults" data-toc-modified-id="Adults-8.8"><span class="toc-item-num">8.8&nbsp;&nbsp;</span><code>Adults</code></a></span></li><li><span><a href="#Children" data-toc-modified-id="Children-8.9"><span class="toc-item-num">8.9&nbsp;&nbsp;</span><code>Children</code></a></span></li><li><span><a href="#Babies" data-toc-modified-id="Babies-8.10"><span class="toc-item-num">8.10&nbsp;&nbsp;</span><code>Babies</code></a></span></li><li><span><a href="#Meal" data-toc-modified-id="Meal-8.11"><span class="toc-item-num">8.11&nbsp;&nbsp;</span><code>Meal</code></a></span></li><li><span><a href="#Country" data-toc-modified-id="Country-8.12"><span class="toc-item-num">8.12&nbsp;&nbsp;</span><code>Country</code></a></span></li><li><span><a href="#Market_Segment" data-toc-modified-id="Market_Segment-8.13"><span class="toc-item-num">8.13&nbsp;&nbsp;</span><code>Market_Segment</code></a></span></li><li><span><a href="#Is_Repeated_Guest" data-toc-modified-id="Is_Repeated_Guest-8.14"><span class="toc-item-num">8.14&nbsp;&nbsp;</span><code>Is_Repeated_Guest</code></a></span></li><li><span><a href="#Previous_Cancellations" data-toc-modified-id="Previous_Cancellations-8.15"><span class="toc-item-num">8.15&nbsp;&nbsp;</span><code>Previous_Cancellations</code></a></span></li><li><span><a href="#Previous_Bookings_Not_Canceled" data-toc-modified-id="Previous_Bookings_Not_Canceled-8.16"><span class="toc-item-num">8.16&nbsp;&nbsp;</span><code>Previous_Bookings_Not_Canceled</code></a></span></li><li><span><a href="#Reserved_Room_Type" data-toc-modified-id="Reserved_Room_Type-8.17"><span class="toc-item-num">8.17&nbsp;&nbsp;</span><code>Reserved_Room_Type</code></a></span></li><li><span><a href="#Assigned_Room_Type" data-toc-modified-id="Assigned_Room_Type-8.18"><span class="toc-item-num">8.18&nbsp;&nbsp;</span><code>Assigned_Room_Type</code></a></span></li><li><span><a href="#Booking_Changes" data-toc-modified-id="Booking_Changes-8.19"><span class="toc-item-num">8.19&nbsp;&nbsp;</span><code>Booking_Changes</code></a></span></li><li><span><a href="#Deposit_Type" data-toc-modified-id="Deposit_Type-8.20"><span class="toc-item-num">8.20&nbsp;&nbsp;</span><code>Deposit_Type</code></a></span></li><li><span><a href="#Days_in_Waiting_List" data-toc-modified-id="Days_in_Waiting_List-8.21"><span class="toc-item-num">8.21&nbsp;&nbsp;</span><code>Days_in_Waiting_List</code></a></span></li><li><span><a href="#Customer_Type" data-toc-modified-id="Customer_Type-8.22"><span class="toc-item-num">8.22&nbsp;&nbsp;</span><code>Customer_Type</code></a></span></li><li><span><a href="#ADR" data-toc-modified-id="ADR-8.23"><span class="toc-item-num">8.23&nbsp;&nbsp;</span><code>ADR</code></a></span></li><li><span><a href="#Required_Car_Parking_Spaces" data-toc-modified-id="Required_Car_Parking_Spaces-8.24"><span class="toc-item-num">8.24&nbsp;&nbsp;</span><code>Required_Car_Parking_Spaces</code></a></span></li><li><span><a href="#Total_of_Special_Requests" data-toc-modified-id="Total_of_Special_Requests-8.25"><span class="toc-item-num">8.25&nbsp;&nbsp;</span><code>Total_of_Special_Requests</code></a></span></li><li><span><a href="#Reservation_Status_Date" data-toc-modified-id="Reservation_Status_Date-8.26"><span class="toc-item-num">8.26&nbsp;&nbsp;</span><code>Reservation_Status_Date</code></a></span></li><li><span><a href="#Agent_Group" data-toc-modified-id="Agent_Group-8.27"><span class="toc-item-num">8.27&nbsp;&nbsp;</span><code>Agent_Group</code></a></span></li></ul></li><li><span><a href="#🛑-Post-EDA" data-toc-modified-id="🛑-Post-EDA-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>🛑 <strong>Post-EDA</strong></a></span><ul class="toc-item"><li><span><a href="#Reservation_Status:-Converting-No-Show-to-Canceled" data-toc-modified-id="Reservation_Status:-Converting-No-Show-to-Canceled-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span><code>Reservation_Status</code>: Converting <code>No-Show</code> to <code>Canceled</code></a></span><ul class="toc-item"><li><span><a href="#Review---Reservation_Status" data-toc-modified-id="Review---Reservation_Status-9.1.1"><span class="toc-item-num">9.1.1&nbsp;&nbsp;</span>Review - <code>Reservation_Status</code></a></span></li></ul></li><li><span><a href="#Engineering-Arrival_Date" data-toc-modified-id="Engineering-Arrival_Date-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Engineering <code>Arrival_Date</code></a></span></li><li><span><a href="#Dropping-Reservation_Status" data-toc-modified-id="Dropping-Reservation_Status-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Dropping <code>Reservation_Status</code></a></span></li><li><span><a href="#Dropping-Arrival_Date_Year" data-toc-modified-id="Dropping-Arrival_Date_Year-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span>Dropping <code>Arrival_Date_Year</code></a></span></li><li><span><a href="#Final-Data-Review" data-toc-modified-id="Final-Data-Review-9.5"><span class="toc-item-num">9.5&nbsp;&nbsp;</span>Final Data Review</a></span></li></ul></li><li><span><a href="#Of-Pandas-and-Pickles" data-toc-modified-id="Of-Pandas-and-Pickles-10"><span class="toc-item-num">10&nbsp;&nbsp;</span><strong>Of Pandas and Pickles</strong></a></span><ul class="toc-item"><li><span><a href="#Peter-Panda-Picked-a-Peck-of-Pickled-DataFrames..." data-toc-modified-id="Peter-Panda-Picked-a-Peck-of-Pickled-DataFrames...-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Peter Panda Picked a Peck of Pickled DataFrames...</a></span></li></ul></li><li><span><a href="#Moving-to-Modeling!" data-toc-modified-id="Moving-to-Modeling!-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Moving to Modeling!</a></span></li></ul></div>

# **Import Packages**

---

> To start off, I will import a variety of packages to assist with handling my data; creating visualizations; and reviewing statistical data.

---

In [ ]:
## Data Handling
import pandas as pd
import numpy as np

## Visualizations
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

## Custom-made Functions
from bmc_functions import eda

In [ ]:
## Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('max_rows', 50)
%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 2

# **Read Data**

In [ ]:
## Reading data
source = './data/hotel_bookings.pickle'
data = pd.read_pickle(source, compression = "gzip")
data

# **Identifying Target Feature**

---

> For my classification analysis, **I will use the `is_canceled` feature as my target feature.** This feature indicates whether a reservation was canceled (0 = check-out, 1= canceled).
>
> There is another feature, `reservation_status`, that also looks valuable. I will compare that feature against `is_canceled` to investigate any differences between the two.

---

In [ ]:
## Inspecting target feature
data['is_canceled'].describe()

In [ ]:
data['is_canceled'].value_counts(normalize=True, dropna=False)

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

ax = data['is_canceled'].value_counts(1, dropna=False).plot(kind='bar',ax=ax)

ax.set(xlabel = 'Status', ylabel= 'Percentages')

plt.xticks([0, 1], ['Checked-Out', 'Cancelled'], rotation=0)
plt.yticks([.1, .2, .3, .4, .5, .6], [10, 20, 30, 40, 50, 60])
plt.suptitle('Breakdown of Reservation Statuses')

ax.set_facecolor('0.9')
fig.set_facecolor('0.975')
plt.savefig('./img/cxl_stat.png',transparent=False, bbox_inches='tight',
           dpi=300)
plt.show()
plt.close()

---

> Based on this initial review, I see there is a moderate class imbalance between whether a reservation canceled favoring non-canceled reservations with a 63%/37% split between not-canceled/canceled, respectively.
>
> I will keep this imbalance in mind when I perform my modeling in my next notebook. Imbalanced classes may have a significant negative impact on a model's performance; I will need to address the imbalance at that time.

---

# Visualizing the Data

In [ ]:
clean_x_label = 'arrival_date_year'.replace('_', ' ').title()
clean_x_label

In [ ]:
ax = sns.countplot(x = data['arrival_date_year'])
ax.set(xlabel=clean_x_label, ylabel = 'Reservation Count')
plt.suptitle('Years')
plt.tight_layout()
plt.show()
plt.close();

# **Reviewing Statistics**

In [ ]:
## Sorting report by number of missing values
eda.report_df(data).sort_values('null_sum', ascending=False).style\
                                       .background_gradient(subset='null_pct')

---

> **Notes**

---

---

**Reviewing Reports - Missing Values**

> Based on the post-split results, I see that both dataframes are missing values for `company,` `agent`, `country`, and `children`.
>
> ***Special note:*** As noted in the data's documentation (located in *"details.md"*), any missing values are intentional representations of features that were not applicable to a reservation.
---

**`Company` and `Agent` Features**

>* `company:` 94%
>* `agent:` 14%
>
> Due to the large number of missing values for `company`, **I will drop the `company` feature.**
>
> The `agent` feature shows a significant number of missing values while including a very large number of unique values. **I will need to review this feature in more depth to determine whether or not to keep it for modeling.**

**`Country` and `Children` Features**

> The remaining two features with missing values are `country` and `children`.
>
> **As there are a small number of missing values in both features, I will keep them and fill the missing values via a SimpleImputer in my modeling pipeline.** By incorporating a SimpleImputer in my pipeline, I will be able to determine the best method to determine the value to fill my missing values. 

---

## Dropping `Company` Column

In [ ]:
# Dropping "company" column (95% missing values)
data.drop(columns = ['company'], inplace=True)
data

In [ ]:
## Confirming 'company' removal from dataframe
'company' not in data

## Inspecting the `Agent` Feature

---

> Large number of unique features, all categorical/unique identifiers.
>
> Creates a problem for OHE to prep for modeling - creates 300+ new features to model

---

In [ ]:
## Confirming number of unique values
data['agent'].nunique()

In [ ]:
## Reviewing top ten agents by percentage of bookings
data['agent'].value_counts(normalize=True, ascending=False, dropna=False).iloc[:10]

In [ ]:
## Visualizing top ten agents by percentage of bookings
fig, ax = plt.subplots()
data['agent'].value_counts(normalize=True, ascending=True, dropna=False)\
                                           .iloc[-10:].plot(kind='barh', ax=ax)
fig.suptitle('Top Ten Agents by Volume')
ax.set_xlabel("Percentage of Total Reservations")
ax.set_ylabel("Agent ID");

---

> Compress to 4 total classes  - top three and the rest
>
> Top three (non-NaN) represent 45% all data
>
> Becomes 4-class classification, reducing dimensionality while keeping data

---

In [ ]:
## Converting non-top-3 ID values to placeholder "999"
cond = [data['agent'] == 9.00,
        data['agent'] == 240.00,
        data['agent'] == 1.00]

choice = [data['agent'], data['agent'], data['agent']]

data['agent_group'] = np.select(cond, choice, 999)
data['agent_group']

In [ ]:
data[['agent', 'agent_group']]

In [ ]:
## Creating new Series for visualization
new_cats = data['agent_group'].value_counts(1, ascending=True)
new_cats

In [ ]:
##Visualizing new feature

fig, ax = plt.subplots(figsize= (8, 4))
new_cats.plot(kind='barh', ax=ax)

fig.suptitle('Agent - New Categories')

ax.set_xlabel("Percentage of Total Reservations")
ax.set_xticks([0, .1, .2, .3, .4, .5])
ax.set_xticklabels(['0', '10','20', '30', '40', '50'])
ax.set_ylabel("Agent ID")

for i,vc in enumerate(new_cats):
    plt.text(x=vc, y=i, s=f"{vc:.0%}")

plt.tight_layout();

In [ ]:
# Dropping "agent" feature after conversion - try/except to prevent KeyErrors

try:
    data.drop(columns = ['agent'], inplace=True)
except:
    pass

data.columns

In [ ]:
## Confirming 'agent' removal from dataframe
'agent' not in data

In [ ]:
## Deleting variables to free up space
try:
    del cond, choice, new_cats
except:
    pass

---

> Created new feature to condense values into one of four categories: one of top three agents by volume and then the rest.

---

## Comparing `Market_Segment` and `Distribution_Channel`

---

> What are the differences between the values in these features?
>
> Is one feature more descriptive than the other?

---

In [ ]:
data['distribution_channel'].value_counts(1)

In [ ]:
data['market_segment'].value_counts(1)

---

> Both features describe similar information.
>
> `Market_segment` is more descriptive; will keep mkt seg and drop dist chnl

---

In [ ]:
## Dropping "distribution_channel"
data.drop(columns = 'distribution_channel', inplace=True)

In [ ]:
## Confirming 'distribution_channel' removal from dataframe
'distribution_channel' not in data

## Filling Missing Values for `Country` and `Children`

---

> As there are so few missing values for the country and children features, I will impute the most frequent values for each feature.
>
> **I do not expect the imputation method to effect my future modeling results.** If the number of missing values was more substantial, I would incorporate a `SimpleImputer` in my future modeling pipeline.

---

In [ ]:
## Identify columns with missing data
nan_list = list(data.isna().sum()[data.isna().sum() > 0].index)
nan_list

In [ ]:
## Impute the most frequent value for each column
for col in nan_list:
    data[col].fillna(data[col].mode()[0], inplace=True)

In [ ]:
## Confirming there are no remaining missing values
for col in nan_list:
    display(data[col].value_counts(1, dropna=0))
    print(f'Total missing values for {col.title()}: {data[col].isna().sum()}')

In [ ]:
## Deleting variables to free up space
try:
    del nan_list, col
except:
    pass

# **Inspecting Feature Data Types**

---

>

---

In [ ]:
## Inspecting dataypes for "data"
data.dtypes.sort_values()

---

**Review - Datatypes**

> After reviewing the data types, I noticed `agent_group` should be changed to the `string` datatype. This feature represents unique identifiers for booking agents and need to be treated as categorical data.
>
> **Convert to `string` type:**
> * `agent_group`
>
> **Convert to `datetime` type:**
> * `reservation_status_date`

---

## Converting to String

In [ ]:
## Converting column to string

data.loc[:,'agent_group'] = data.loc[:,'agent_group'].astype(int)
data.loc[:,'agent_group'] = data.loc[:,'agent_group'].astype(str)
print(f'Datatype: {data["agent_group"].dtype}')

## Converting to DateTime

In [ ]:
## Converting to datetime
data['reservation_status_date'] = pd.to_datetime(data['reservation_status_date'])
print(f'Datatype: {data["reservation_status_date"].dtype}')

# **EDA - Features**

---

**In-Depth EDA per Feature**

> Now that I reviewed my missing values and confirmed my datatypes, I will inspect the details of each of my features.

---
**Note:**

> DataFrame styling code used in `explore_feature()` function adapted from this [source](https://stackoverflow.com/questions/59769161/python-color-pandas-dataframe-based-on-multiindex#:~:text=2-,You,-can%20use%20Styler).

---

# -- > 🛑 **FIX**: Update narrative per feature

## **Toggle Visualizations**

In [ ]:
## Boolean setting to control whether to show the EDA visualizations
show_visualization = False

## `Reservation_Status`

---

> Text

---

In [ ]:
## Reviewing details for reservation_status
eda.explore_feature(data,'reservation_status',plot_label ='Status',
                    plot_title= 'Reservation Status',
                    show_visualization = show_visualization);

---

**Feature Review**

> `Reservation_status` closely mirrors the values for my target feature, with some slight differences due to "no-show" values. **To prepare it for modeling, I will combine the `No-Show` status and `Canceled` values.**

**Actions**

>For the purposes of my analysis, **I will treat `No-Show` reservations as `Canceled` reservations** due to their limited number preventing me from effectively using it as a third class.

****

> The most notable difference between the city and resort hotels would be the number of cancellations: *The city hotel shows a much larger proportion of canceled reservations vs. the resort hotel.* 
* This may be due to a variety of factors, including resort guests booking when they are more certain of their plans or the resort hotel may charge a cancellation fee.
>
> No-Show reservations are low for both hotels, supporting my decision to merge no-shows with cancellations. 

---

## `Is_Canceled`

---

**test**

---

In [ ]:
## Reviewing details for 'is_canceled'
eda.explore_feature(data,'is_canceled', 
                    
                    normalize=False,
                    plot_label ='Cancellation Status',
                    plot_title= 'Reservation Status',
                    show_visualization = show_visualization);

---

**Feature Review**

> After reviewing the results post-"no-show" conversion, `Is_canceled` is a binarization of the `reservation_status`. Reservations are indicated as cancellations if they either cancel or are marked as a "no-show" reservation.

**Actions**

> This feature is a better target feature as the values are already binarized and match the `reservation_status` feature for all of the reservations.
>
> **I will use `is_canceled` in place of the `reservation_status` feature as my target feature.**

****

> The breakdown between hotels is the same as `reservation_status` and confirms that the resort hotel experiences fewer cancellations vs. the city hotel.

---

## `Lead_Time`

---

**test**

---

In [ ]:
## Reviewing details for 'lead_time'
eda.explore_feature(data,'lead_time',bins = 5, marginal = 'box',
                    plot_label ='Number of Days',
                    plot_title= 'Lead Time (Days)',
                    show_visualization = show_visualization);

---

**Feature Review**

> `Lead_Time` indicates how far in advance reservations are booked in days. *This information is particularly useful in hospitality for Revenue Management (RM) and Operations (Ops).*
>
>  * RM needs to know **when to expect bookings** and **when to monitor rates and availability** closely to make any necessary changes to optimize revenue.
>
>
>  * Ops uses this information to **forecast how many reservations will book in a short-term booking window** (I usually focused on 0-3 days prior to arrival).
>
> * **This forecast is critical to determine staffing and supplies in particular** - when building our schedules, we consider the current number of booked reservations and the forecasted bookings to determine how many staff members to schedule and if we have enough supplies, etc..
>  * *Being the only staff member at the Front Desk during a rush of arrivals due to a snow storm is NOT fun!*

**Actions**

> I noticed there are a significant number of outliers for both properties. **I will remove the outliers based on the z-score percentiles prior to modeling.**

****

> The histograms and box plots for both hotels match up closely, but it is clear that **the city hotel has a larger range of lead times for cancellations vs. the resort hotel.**

---

## `Arrival_Date_Year`

---

**test**

---

In [ ]:
## Reviewing details for 'arrival_date_year'
eda.explore_feature(data,'arrival_date_year',marginal = 'box',
                    plot_label ='Year',
                    plot_title= 'Arrival Date (Year)',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER 

**Actions**

> This feature is not useful for my future modeling, so I will drop this feature at the end of my EDA process.

---


## `Stays_in_Weekend_Nights`

---

**City**

---

In [ ]:
## Reviewing details for 'stays_in_weekend_nights'
eda.explore_feature(data,'stays_in_weekend_nights',bins = 5,
                    marginal = 'box',
                    plot_label ='Number of Days',
                    plot_title= 'Lead Time (Days)',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Stays_in_Week_Nights`

---

**test**

---

In [ ]:
## Reviewing details for 'stays_in_week_nights'
eda.explore_feature(data,'stays_in_week_nights',bins = 5,
                    marginal = 'box',
                    plot_label ='Number of Prior Stays',
                    plot_title= 'Stays in Week Nights',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Adults`

---

**test**

---

In [ ]:
## Reviewing details 'adults'
eda.explore_feature(data,'adults',bins = 3,
                    plot_label ='Number of Adults',
                    plot_title= 'Adults',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Children`

---

**data**

---

In [ ]:
## Reviewing details for 'children'
eda.explore_feature(data,'children',bins = 5,
                    
                    plot_label ='Number of Children',
                    plot_title= 'Children',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Babies`

---

**data**

---

In [ ]:
## Reviewing details - 'babies'
eda.explore_feature(data,'babies',bins = 5,
                    
                    plot_label ='Number of Babies',
                    plot_title= 'Babies',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Meal`

---

**data**

---

In [ ]:
## Reviewing details for - 'meal'
eda.explore_feature(data,'meal',plot_label ='Types of Meal',
                    plot_title= 'Meal',show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Country`

---

**City**

---

In [ ]:
## Reviewing details for 'country'
eda.explore_feature(data,'country',marginal = 'box',normalize=False,
                    plot_label ='Country',
                    plot_title= 'Country',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Market_Segment`

---

**City**

---

In [ ]:
## Reviewing details for - 'market_segment'
eda.explore_feature(data,'market_segment',marginal = 'box',
                    plot_label ='Market Segment',
                    plot_title= 'Market Segment',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Is_Repeated_Guest`

---

**City**

---

In [ ]:
## Reviewing details for 'is_repeated_guest'
eda.explore_feature(data,'is_repeated_guest',
                    plot_label =' Repeat Guest',
                    plot_title= ' Repeat Guest',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Previous_Cancellations`

---

**City**

---

In [ ]:
## Reviewing details for 'previous_cancellations'
eda.explore_feature(data,'previous_cancellations',bins = 5,
                    normalize=False,
                    plot_label ='Number of Cancellations',
                    plot_title= 'Previous Cancellations',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Previous_Bookings_Not_Canceled`

---

**City**

---

In [ ]:
## Reviewing details for 'previous_bookings_not_canceled'
eda.explore_feature(data,'previous_bookings_not_canceled',
                    bins = 5,marginal = 'box',
                    plot_label ='Number of Bookings Not Canceled',
                    plot_title= 'Previous Bookings Not Canceled',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Reserved_Room_Type`

---

**City**

---

In [ ]:
## Reviewing details for - 'reserved_room_type'
eda.explore_feature(data,'reserved_room_type',
                    
                    plot_label ='Reserved Toom Type',
                    plot_title= 'Reserved Toom Type',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Assigned_Room_Type`

---

**City**

---

In [ ]:
## Reviewing details for 'assigned_room_type'
eda.explore_feature(data,'assigned_room_type',
                    
                    plot_label ='Assigned Room Type',
                    plot_title= 'Assigned Room Type',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Booking_Changes`

---

**City**

---

In [ ]:
## Reviewing details for 'booking_changes'
eda.explore_feature(data,'booking_changes',bins = 5,
                    
                    plot_label ='booking_changes',
                    plot_title= 'booking_changes',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Deposit_Type`

---

**City**

---

In [ ]:
## Reviewing details for 'deposit_type'
eda.explore_feature(data,'deposit_type',normalize=False,
                    plot_label ='Deposit Type',
                    plot_title= 'Deposit Type',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Days_in_Waiting_List`

---

**City**

---

In [ ]:
## Reviewing details for 'days_in_waiting_list'
eda.explore_feature(data,'days_in_waiting_list',bins = 5,
                    normalize=False,
                    plot_label ='Days in Waiting List',
                    plot_title= 'Days in Waiting List',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Customer_Type`

---

**City**

---

In [ ]:
## Reviewing details for 'customer_type'
eda.explore_feature(data,'customer_type',marginal = 'box',
                    plot_label ='Customer Type',
                    plot_title= 'Customer Type',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `ADR`

---

**City**

---

In [ ]:
## Reviewing details for 'adr'
eda.explore_feature(data ,'adr',bins = 5,
                    plot_label ='ADR (€)',
                    plot_title= 'ADR (€)',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Required_Car_Parking_Spaces`

---

**City**

---

In [ ]:
## Reviewing details for 'required_car_parking_spaces'
eda.explore_feature(data,'required_car_parking_spaces',bins = 5,
                    normalize=False,
                    plot_label ='Required Car Parking Spaces',
                    plot_title= 'Required Car Parking Spaces',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Total_of_Special_Requests`

---

**City**

---

In [ ]:
## Reviewing details for 'total_of_special_requests'
eda.explore_feature(data,'total_of_special_requests',bins = 5,
                    marginal = 'box',
                    plot_label ='Total of Special Requests',
                    plot_title= 'Total of Special Requests',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Reservation_Status_Date`

---

**City**

---

In [ ]:
## Reviewing details for 'reservation_status_date'
eda.explore_feature(data,'reservation_status_date',marginal = 'box',
                    bins=3,
                    plot_label ='Reservation Status Date',
                    plot_title= 'Reservation Status Date',
                    show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## `Agent_Group`

---

>

---

In [ ]:
## Reviewing details for 'agent_group'
eda.explore_feature(data,'agent_group', plot_label ='Booking Agent Group',
                    plot_title= 'Agent Group', show_visualization = show_visualization);

---

**Feature Review**

> PLACEHOLDER

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

# 🛑 **Post-EDA**

---

**FIX/UPDATE ALL OF THE NARRATIVE ELEMENTS OF THE REST OF THIS CODE.**

---

---

**Finishing Touches**

> Now that I reviewed all of my features; confirmed there are no missing values; and confirmed all of the datatypes are correct, I will finish the remaining preprocessing.

**Outliers**

> Based on my EDA, I noticed several features show significant outliers. If I kept these outlying data points, they could have a negative impact on my future models' performances.

**Process**
> I will first identify the features with outliers; then use the z-scores of each data point to determine the outliers. Any absolute-valued z-score greater than 3 will be considered an outlier and will be disregarded.

---

## Updating `Meal`

---

> The feature reference dictionary states that the "undefined" values for `meal` are part of the "SC" category. While there are not many "undefined" entries, I feel that it would be most accurate to put them all together to be most accurate.

---

In [ ]:
data['meal'].value_counts(1)

In [ ]:
data['meal'].replace('Undefined', 'SC').value_counts(1)

In [ ]:
data['meal'] = data['meal'].replace('Undefined', 'SC')
data['meal'].value_counts(1)

In [ ]:
## Reviewing details for arrival_day
eda.explore_feature(data,'meal',plot_label ='Meal Type',
                    plot_title= 'Meals',
                    show_visualization = show_visualization);

## Engineering `Arrival_Date`

---

**City**

---

In [ ]:
## Converting from month, day of month, and year to a single datetime column
data['arrival_date'] = data['arrival_date_month'] +' '+ \
                                data['arrival_date_day_of_month']\
                                .astype(str) +', '+ \
                                data['arrival_date_year'].astype(str)
data['arrival_date'] = pd.to_datetime(data['arrival_date'])
data['arrival_date']

In [ ]:
## Determining the day of the week of arrival 
data.loc[:,'arrival_day'] = data.loc[:,'arrival_date'].dt.day_name()
data['arrival_day']

In [ ]:
## Reviewing results
data[['arrival_day', 'arrival_date']]

In [ ]:
## Reviewing details for arrival_day
eda.explore_feature(data,'arrival_day',plot_label ='Day',
                    plot_title= 'Arrival Day',
                    show_visualization = show_visualization);

---

**Feature Review**

> I created this new feature to merge the arrival year/month/day-of-month features into one usable feature. 

**Actions**

> PLACEHOLDER

****

> PLACEHOLDER

---

## Engineering `Stay Length`

---

> One major feature of a guest's reservation is missing from the original dataset: the overall length of stay (LoS). I feel a reservation's LoS is relevant to my analysis, and so I will engineer this feature by adding the number of week day and weekend nights together.

---

In [ ]:
los = data['stays_in_weekend_nights'] + data['stays_in_week_nights']
los.value_counts(dropna=False)

In [ ]:
data['stay_length'] = los
data['stay_length']

In [ ]:
## Reviewing details for stay_length
eda.explore_feature(data,'stay_length',plot_label ='Length',
                    plot_title= 'Length of Stay', bins=5,
                    show_visualization = show_visualization);

## Dropping `Reservation_Status`

---

> 

---

In [ ]:
## Dropping "reservation_status"
data.drop(columns = 'reservation_status', inplace=True)

In [ ]:
## Confirming 'reservation_status' removal from dataframe
'reservation_status' not in data

## Dropping `Arrival_Date_Year`

---

> 

---

In [ ]:
## Dropping "arrival_date_year"
data.drop(columns = 'arrival_date_year', inplace=True)

In [ ]:
## Confirming 'arrival_date_year' removal from dataframe
'arrival_date_year' not in data

## Final Data Review

---

> 

---

In [ ]:
data

# **Of Pandas and Pickles**

---

> Now I am ready to save the cleaned and processed data for modeling in my next notebook.
>
> In order to preserve the datatypes and details of my data, I will use the "Pickle" module to serialize the data and save four files - one for each dataframe (two hotels; filtered/not).
>
>**First**, I will add unique names to each of my dataframe indices. **Then**, I will pickle the files. Finally, I will reopen the pickled files in my next notebook.

---

## Peter Panda Picked a Peck of Pickled DataFrames...

> 

In [ ]:
## Pickling with Pandas
data.to_pickle(path = './data/data_prepped.pickle',
            compression = 'gzip')
print(f'Successfully pickled!')

# Moving to Modeling!

---

> Now that I completed the pre-processing and EDA steps, I will move to my next notebook to perform my classification modeling.

---